In [3]:
# ************************************Import lib for workflow
import pandas as pd
import numpy as np
import os
#import re
# pyproj and utm for Lat Long conversion
#from pyproj import Proj
import utm

In [68]:
from dataclasses import dataclass, field

In [69]:
@dataclass
class DirectionalSurvey:
    """
    Dataclass for Production Monthly, takes a list of dictionaries and converts them into np.arrays
    Intended to be run per well.

    Args:
    wellId (required)
    fill in info for the rest of the schema
    Other (dict of all other columns in data set that do not fall into the dataclass schema)

    Returns:
    dataclass Production Monthly object
    """

    # TODO: when a default value is set to none it creates array(None), needs to be array(None,None,ect...)
    # how is the none supposed to work within the data class array.

    wellId: np.array
    md: np.array
    inc: np.array
    azim: np.array
    tvd: np.array = field(default=None, metadata={'unit': 'float'})
    n_s_deviation: np.array = field(default=None, metadata={'unit': 'float'})
    n_s: np.array = field(default=None, metadata={'unit': 'float'})
    e_w_devitaion: np.array = field(default=None, metadata={'unit': 'float'})
    e_w: np.array = field(default=None, metadata={'unit': 'float'})
    surface_latitude: np.array = field(default=None, metadata={'unit': 'float'})
    surface_longitude: np.array = field(default=None, metadata={'unit': 'float'})

In [70]:
class Survey:
    """
    Get information about a well from raw json files
    reformat raw production monthly data
    Bokeh Display:
    display map chart of well location
    display line chart of well production
    """

    def __init__(self, directional_survey_data):
        """
        Well object with a wells well_info (one line well header data)
        and corresponding well_prod_monthly well production data

        Attributes:
        well_info (Dataclass Object) WellInfo well object
        well_prod_monthly (Dataclass Object) ProductionMonthly object
        """
        #directional_survey_data = WellInfo(**json_well_info)
        #well_prod_monthly = get_production_monthly_dataclass(json_well_prod_monthly)

        #directional_survey_points = get_directional_survey_dataclass(directional_survey_data)
        
        
        
        self.directional_survey_points = directional_survey_points
        
        

In [71]:
file = "C:/Users/BpAmos/Documents/repos/directional-survey-converter/data/Well_A_Dir_Survey.csv"

df = pd.read_csv(file, sep=',')
#dfwells = dfwells[['UWI','Measured Depth','Deviation Angle','Deviation Azimuth','Deviation N/S',]]
#df.tail()

In [72]:
file = "C:/Users/BpAmos/Documents/repos/directional-survey-converter/data/well_surface_location.csv"

df_surface = pd.read_csv(file, sep=',')
df_surface.tail()

df = pd.merge(df,df_surface,on="UWI")
df.tail()


,UWI,Measured Depth,TV Depth,Deviation Angle,Deviation Azimuth,Deviation N/S,N/S,Deviation E/W,E/W,PRODFit Formation Code,PRODFit Formation Name,Point Type Code,Point Type Name,Surface Latitude,Surface Longitude
36,Well_A,9099,8023.49,91.1,226.5,907.19,S,891.66,W,NaN,NaN,NaN,NaN,33.438389,-97.482414
37,Well_A,9161,8021.59,92.4,226.1,950.00,S,936.47,W,NaN,NaN,NaN,NaN,33.438389,-97.482414
38,Well_A,9224,8019.34,91.7,226.5,993.50,S,981.98,W,NaN,NaN,NaN,NaN,33.438389,-97.482414
39,Well_A,9285,8017.05,92.6,226.1,1035.62,S,1026.05,W,NaN,NaN,NaN,NaN,33.438389,-97.482414
40,Well_A,9328,8015.10,92.6,226.1,1065.40,S,1057.00,W,NaN,NaN,PE,"PROJECTED, END POINT",33.438389,-97.482414


In [73]:
df.rename(columns={'UWI':'wellId',
                   'Measured Depth':'md',
                   'Deviation Angle':'inc',
                   'Deviation Azimuth':'azim',
                   'TV Depth':'tvd',
                   'Deviation N/S':'n_s_deviation',
                   'N/S':'n_s',
                   'Deviation E/W':'e_w_devitaion',
                   'E/W':'e_w',
                   'Surface Latitude':'surface_latitude',
                   'Surface Longitude':'surface_longitude',},inplace=True)

In [74]:
#df.to_json(orient='records')

In [75]:
def get_directional_survey_dataclass(survey_data_list):
    """
    iterates over a list of production data
    looks for field names that match the ProductionMonthly dataclass fields
    converts these key value pairs to np.arrays and puts them in an empty dict
    returns the ProductionMonthly dataclass object for this new dict

    Args:
    production_data_list (list of dicts) list of production data dicts to iterate over
    (data dict should should match required dict format for production dataclass)

    Returns:
    ProductionMonthly dataclass object for one well
    """

    # ProductionMonthly dataclass fields
    survey_fields_list = ['wellId', 'md', 'inc', 'azim', 'tvd',
                              'n_s_deviation', 'n_s', 'e_w_devitaion', 'e_w',
                              'surface_latitude', 'surface_longitude']
    survey_dict = {}
    for field_name in survey_fields_list:
        # get available dict fields from production data
        if field_name in list(survey_data_list[0].keys()):
            # convert data to np.array and update dict
            survey_dict.update({field_name: np.asarray([ROW[field_name] for ROW in survey_data_list])})

    # get ProductionMonthly dataclass object for dict
    directional_survey = DirectionalSurvey(**survey_dict)

    return directional_survey

In [80]:
my_df = df[['wellId','md','inc','azim','surface_latitude', 'surface_longitude']]

In [81]:
mydict = my_df.to_dict(orient='records')


In [82]:
directional_survey_points = get_directional_survey_dataclass(mydict)

In [83]:
directional_survey_points

DirectionalSurvey(wellId=array(['Well_A', 'Well_A', 'Well_A', 'Well_A', 'Well_A', 'Well_A',
       'Well_A', 'Well_A', 'Well_A', 'Well_A', 'Well_A', 'Well_A',
       'Well_A', 'Well_A', 'Well_A', 'Well_A', 'Well_A', 'Well_A',
       'Well_A', 'Well_A', 'Well_A', 'Well_A', 'Well_A', 'Well_A',
       'Well_A', 'Well_A', 'Well_A', 'Well_A', 'Well_A', 'Well_A',
       'Well_A', 'Well_A', 'Well_A', 'Well_A', 'Well_A', 'Well_A',
       'Well_A', 'Well_A', 'Well_A', 'Well_A', 'Well_A'], dtype='<U6'), md=array([7654, 7700, 7743, 7774, 7806, 7838, 7870, 7901, 7932, 7963, 7994,
       8026, 8057, 8089, 8120, 8152, 8186, 8218, 8249, 8279, 8314, 8346,
       8377, 8409, 8440, 8472, 8502, 8564, 8627, 8658, 8720, 8783, 8847,
       8910, 8973, 9036, 9099, 9161, 9224, 9285, 9328]), inc=array([ 2. ,  1.9,  6.5, 12.1, 17.8, 23.6, 29.6, 35.9, 42. , 46.8, 50.7,
       53.1, 57.7, 63.5, 69.2, 75. , 83.8, 90.6, 92.4, 91.5, 91.7, 89.6,
       89.7, 90.3, 90.8, 88.9, 89.4, 88.2, 89.7, 92.2, 91.5, 92.7, 91.8,

In [17]:
my_survey = Survey(df)
#my_survey.directional_survey_points

In [20]:
df = pd.DataFrame({'wellId': my_survey.directional_survey_points.wellId,
                   'md': my_survey.directional_survey_points.md,
                   'inc': my_survey.directional_survey_points.inc,
                   'azim': my_survey.directional_survey_points.azim,
                   'tvd': my_survey.directional_survey_points.tvd,
                   'n_s_deviation': my_survey.directional_survey_points.n_s_deviation,
                   'n_s': my_survey.directional_survey_points.n_s,
                   'e_w_devitaion': my_survey.directional_survey_points.e_w_devitaion,
                   'e_w': my_survey.directional_survey_points.e_w,
                   'surface_latitude': my_survey.directional_survey_points.surface_latitude,
                   'surface_longitude': my_survey.directional_survey_points.surface_longitude,
                  })
df.head()

,wellId,md,inc,azim,tvd,n_s_deviation,n_s,e_w_devitaion,e_w,surface_latitude,surface_longitude
0,Well_A,7654,2.0,256.3,7651.41,92.48,S,0.41,W,33.438389,-97.482414
1,Well_A,7700,1.9,253.3,7697.38,92.90,S,1.94,W,33.438389,-97.482414
2,Well_A,7743,6.5,227.9,7740.26,94.74,S,4.44,W,33.438389,-97.482414
3,Well_A,7774,12.1,238.8,7770.84,97.60,S,8.52,W,33.438389,-97.482414
4,Well_A,7806,17.8,234.9,7801.75,102.15,S,15.40,W,33.438389,-97.482414


In [59]:


#***********************************************FUNCTIONS:

#******************************************Deviation survey functions
# gets UTM coords from lat long
def getUTMs(row):
    """
    DESCRIPTION
    """
    
    tup = utm.from_latlon(row.iloc[0],row.iloc[1])
    return pd.Series(tup[:4])

#gets lat long from UTM coords
def getLatLon(row):
    """
    DESCRIPTION
    """
    
    tup = utm.to_latlon(row.iloc[0],row.iloc[1],row.iloc[2],row.iloc[3])
    return pd.Series(tup[:2])


#***************************************************************************

def createDirSrvy():
    """
    IHS Dev Survey
    Bring in Directional Surveys and preprocessed well header
    create X and Y Offset columns 
    Bring in Surface lat and lon for each well 
    Convert Surface lat and lon to X and Y 
    Add the X and Y Offset to the new Surface X and Y to create adjusted X and Ys 
    Convert the adjusted X and Ys to Lat Lon
    
    May take a while depending on data set size
    """
    
    
    #**************************************
    # PROCESSED WELL HEADER IS REQUIRED
    # find and bring in processed well header
    fileNm = os.path.join(
        DATA_PATH, "Processed", "IHSWellHeader.csv")
    
    datacols = ['UWI','Surface Latitude','Surface Longitude']
    
    # read csv, dropna if present, convert uwi to string
    df = pd.read_csv(fileNm, sep=',',usecols=datacols)
    df = df.dropna()
    df['UWI'] = df['UWI'].apply(str)

    print(f'well header length: {len(df)}.')
    print(f'well header length Drop Duplicates: {len(df.drop_duplicates())}.')
    #**********************************************************
    
    # Grab files and append multiple dev survey files together if available
    dfDirSrvy = appendDfs(DATA_PATH,'xlsx',sheetName='Survey')

    #convert UWI to string
    dfDirSrvy['UWI'] = dfDirSrvy['UWI'].apply(str)

    # calculate the xyOffsets
    dfDirSrvy = calcXYOffset(dfDirSrvy)

    # convert surface x and y to lat and long then add the X and Y offsets from the surveys
    # then convert the adjusted x and y's to lat and long.

    # convert surface x and y to surface lat and long
    df[['Surface_X','Surface_Y','ZoneNumber','ZoneLetter']] = df[['Surface Latitude','Surface Longitude']].apply(getUTMs , axis=1)
    print('converted surface x and y to surface lat and long')

    #merge the new surface x and y and surface lat lon with the deviation dataframe for each uwi
    df = pd.merge(dfDirSrvy, df, on='UWI', how='left')

    # create X and Y columns for each deviation point per uwi
    # add the x and y offset to the surface x and y for each uwi
    df['X_Adjusted'] = df['Surface_X']+(df['X_Offset']*0.3048)
    df['Y_Adjusted'] = df['Surface_Y']+(df['Y_Offset']*0.3048)

    # convert adjusted x and y back to lat long
    df[['Lat_adjusted','Lon_adjusted']] = df[['X_Adjusted','Y_Adjusted','ZoneNumber','ZoneLetter']].apply(getLatLon , axis=1)
    print('converted adjusted x and y back to lat long')

    #drop columns used in lat lon calc
    df.drop(['Y_Adjusted','X_Adjusted','ZoneLetter',
              'ZoneNumber','Surface_Y','Surface_X'], axis=1, inplace=True)


    #export data
    exportPath = DATA_PATH +'Processed/IHSWellDirSrvy.csv'
    df.to_csv(os.path.join(exportPath),index=False)
    print('EXPORTED: IHSWellDirSrvy.csv')
    return df

In [70]:
def calc_xy_offset(df):
    """
    DESCRIPTION
    """
    
    #X_OFFSET is equal to E/W deviation when E is positive and W is negative
    #Y_OFFSET is equal to N/S deviation when N is positive and S is negative
    
    # create dict to map for offset
    offsetDict = {
        "E" : 1,
        "W" : -1,
        "N" : 1,
        "S" : -1
    }
    # create new columns and map dict * the deviations
    df['X_Offset']= df['e_w'].map(offsetDict)*df['e_w_devitaion']
    df['Y_Offset']= df['n_s'].map(offsetDict)*df['n_s_deviation']

    #reset index
    df = df.reset_index(drop=True)
    return df

In [73]:
df = calc_xy_offset(df)
df.head()

,wellId,md,inc,azim,tvd,n_s_deviation,n_s,e_w_devitaion,e_w,surface_latitude,surface_longitude,X_Offset,Y_Offset
0,Well_A,7654,2.0,256.3,7651.41,92.48,S,0.41,W,33.438389,-97.482414,-0.41,-92.48
1,Well_A,7700,1.9,253.3,7697.38,92.90,S,1.94,W,33.438389,-97.482414,-1.94,-92.90
2,Well_A,7743,6.5,227.9,7740.26,94.74,S,4.44,W,33.438389,-97.482414,-4.44,-94.74
3,Well_A,7774,12.1,238.8,7770.84,97.60,S,8.52,W,33.438389,-97.482414,-8.52,-97.60
4,Well_A,7806,17.8,234.9,7801.75,102.15,S,15.40,W,33.438389,-97.482414,-15.40,-102.15


In [74]:


# convert surface x and y to surface lat and long
df[['Surface_X','Surface_Y','ZoneNumber','ZoneLetter']] = df[['surface_latitude','surface_longitude']].apply(getUTMs , axis=1)
print('converted surface x and y to surface lat and long')


converted surface x and y to surface lat and long


In [75]:
df.head()

,wellId,md,inc,azim,tvd,n_s_deviation,n_s,e_w_devitaion,e_w,surface_latitude,surface_longitude,X_Offset,Y_Offset,Surface_X,Surface_Y,ZoneNumber,ZoneLetter
0,Well_A,7654,2.0,256.3,7651.41,92.48,S,0.41,W,33.438389,-97.482414,-0.41,-92.48,641067.861785,3.700918e+06,14,S
1,Well_A,7700,1.9,253.3,7697.38,92.90,S,1.94,W,33.438389,-97.482414,-1.94,-92.90,641067.861785,3.700918e+06,14,S
2,Well_A,7743,6.5,227.9,7740.26,94.74,S,4.44,W,33.438389,-97.482414,-4.44,-94.74,641067.861785,3.700918e+06,14,S
3,Well_A,7774,12.1,238.8,7770.84,97.60,S,8.52,W,33.438389,-97.482414,-8.52,-97.60,641067.861785,3.700918e+06,14,S
4,Well_A,7806,17.8,234.9,7801.75,102.15,S,15.40,W,33.438389,-97.482414,-15.40,-102.15,641067.861785,3.700918e+06,14,S


In [76]:
df['X_Adjusted'] = df['Surface_X']+(df['X_Offset']*0.3048)
df['Y_Adjusted'] = df['Surface_Y']+(df['Y_Offset']*0.3048)

In [77]:
df.tail()

,wellId,md,inc,azim,tvd,n_s_deviation,n_s,e_w_devitaion,e_w,surface_latitude,surface_longitude,X_Offset,Y_Offset,Surface_X,Surface_Y,ZoneNumber,ZoneLetter,X_Adjusted,Y_Adjusted
36,Well_A,9099,91.1,226.5,8023.49,907.19,S,891.66,W,33.438389,-97.482414,-891.66,-907.19,641067.861785,3.700918e+06,14,S,640796.083817,3.700642e+06
37,Well_A,9161,92.4,226.1,8021.59,950.00,S,936.47,W,33.438389,-97.482414,-936.47,-950.00,641067.861785,3.700918e+06,14,S,640782.425729,3.700629e+06
38,Well_A,9224,91.7,226.5,8019.34,993.50,S,981.98,W,33.438389,-97.482414,-981.98,-993.50,641067.861785,3.700918e+06,14,S,640768.554281,3.700615e+06
39,Well_A,9285,92.6,226.1,8017.05,1035.62,S,1026.05,W,33.438389,-97.482414,-1026.05,-1035.62,641067.861785,3.700918e+06,14,S,640755.121745,3.700602e+06
40,Well_A,9328,92.6,226.1,8015.10,1065.40,S,1057.00,W,33.438389,-97.482414,-1057.00,-1065.40,641067.861785,3.700918e+06,14,S,640745.688185,3.700593e+06


In [78]:
df[['Lat_adjusted','Lon_adjusted']] = df[['X_Adjusted','Y_Adjusted','ZoneNumber','ZoneLetter']].apply(getLatLon , axis=1)
print('converted adjusted x and y back to lat long')

df

converted adjusted x and y back to lat long


,wellId,md,inc,azim,tvd,n_s_deviation,n_s,e_w_devitaion,e_w,surface_latitude,...,X_Offset,Y_Offset,Surface_X,Surface_Y,ZoneNumber,ZoneLetter,X_Adjusted,Y_Adjusted,Lat_adjusted,Lon_adjusted
0,Well_A,7654,2.0,256.3,7651.41,92.48,S,0.41,W,33.438389,...,-0.41,-92.48,641067.861785,3.700918e+06,14,S,641067.736817,3.700890e+06,33.438135,-97.482419
1,Well_A,7700,1.9,253.3,7697.38,92.90,S,1.94,W,33.438389,...,-1.94,-92.90,641067.861785,3.700918e+06,14,S,641067.270473,3.700890e+06,33.438134,-97.482424
2,Well_A,7743,6.5,227.9,7740.26,94.74,S,4.44,W,33.438389,...,-4.44,-94.74,641067.861785,3.700918e+06,14,S,641066.508473,3.700889e+06,33.438129,-97.482433
3,Well_A,7774,12.1,238.8,7770.84,97.60,S,8.52,W,33.438389,...,-8.52,-97.60,641067.861785,3.700918e+06,14,S,641065.264889,3.700888e+06,33.438121,-97.482446
4,Well_A,7806,17.8,234.9,7801.75,102.15,S,15.40,W,33.438389,...,-15.40,-102.15,641067.861785,3.700918e+06,14,S,641063.167865,3.700887e+06,33.438109,-97.482469
5,Well_A,7838,23.6,231.8,7831.67,108.93,S,24.44,W,33.438389,...,-24.44,-108.93,641067.861785,3.700918e+06,14,S,641060.412473,3.700885e+06,33.438091,-97.482499
6,Well_A,7870,29.6,230.4,7860.27,117.94,S,35.57,W,33.438389,...,-35.57,-117.94,641067.861785,3.700918e+06,14,S,641057.020049,3.700882e+06,33.438066,-97.482536
7,Well_A,7901,35.9,228.6,7886.33,128.84,S,48.30,W,33.438389,...,-48.30,-128.84,641067.861785,3.700918e+06,14,S,641053.139945,3.700879e+06,33.438037,-97.482578
8,Well_A,7932,42.0,226.1,7910.43,142.06,S,62.61,W,33.438389,...,-62.61,-142.06,641067.861785,3.700918e+06,14,S,641048.778257,3.700875e+06,33.438001,-97.482625
9,Well_A,7963,46.8,225.1,7932.57,157.23,S,78.10,W,33.438389,...,-78.10,-157.23,641067.861785,3.700918e+06,14,S,641044.056905,3.700870e+06,33.437960,-97.482677
